# Análisis de Texto con spaCy
## Práctica: Búsqueda de Patrones en Percy Jackson

Este notebook demuestra técnicas de Procesamiento de Lenguaje Natural (NLP) usando spaCy para:
- Descargar y procesar texto de libros
- Tokenizar y analizar documentos
- Buscar patrones específicos usando Matcher
- Extraer contexto alrededor de coincidencias

## 1. Configuración Inicial

Verificamos si estamos en Google Colab para descargar dependencias si es necesario.

In [1]:
import sys
import warnings

warnings.filterwarnings('ignore')

IN_COLAB = 'google.colab' in sys.modules

In [2]:
if IN_COLAB:
    !wget https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt
    %pip install -r requirements.txt

## 2. Cargar spaCy

Cargamos el modelo de inglés `en_core_web_sm` que incluye:
- Tokenizador
- Part-of-Speech (POS) tagger
- Dependency parser
- Named Entity Recognizer (NER)

In [3]:
# RUN THIS CELL to perform standard imports:
import spacy
nlp = spacy.load('en_core_web_sm')

## 3. Descargar Dataset

Descargamos los libros de Percy Jackson desde Kaggle y copiamos los archivos a la carpeta actual para facilitar su acceso.

In [4]:
import kagglehub
import os
import shutil

# Download latest version
path = kagglehub.dataset_download("shobhit043/percy-jackson-first-5-books")

print("Path to dataset files:", path)
print("Files downloaded:", os.listdir(path))

# Copy files to current directory
for file in os.listdir(path):
    src = os.path.join(path, file)
    dst = os.path.join(".", file)
    if os.path.isfile(src):
        shutil.copy(src, dst)
    elif os.path.isdir(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)

print("Files in current directory:", os.listdir('.'))

Path to dataset files: C:\Users\apapa\.cache\kagglehub\datasets\shobhit043\percy-jackson-first-5-books\versions\1
Files downloaded: ['percy_jackson_book_1.txt', 'percy_jackson_book_2.txt', 'percy_jackson_book_3.txt', 'percy_jackson_book_4.txt', 'percy_jackson_book_5.txt']
Files in current directory: ['1-spacy-basics.ipynb', '2-tokenization.ipynb', '3-stemming.ipynb', '4-lemmatization.ipynb', '5-vocabulary.ipynb', '6-practice.ipynb', '7-sentiment-analysis.ipynb', 'moviereviews.tsv', 'owlcreek.txt', 'percy_jackson_book_1.txt', 'percy_jackson_book_2.txt', 'percy_jackson_book_3.txt', 'percy_jackson_book_4.txt', 'percy_jackson_book_5.txt', 'reaganomics.txt', 'sentimentdataset.csv', 'Work 6 practice.ipynb']


## 4. Procesar el Documento

Leemos el primer libro de Percy Jackson y lo procesamos con spaCy. El objeto `doc` contiene:
- Tokens individuales
- Información gramatical (POS tags, dependencias)
- Entidades nombradas
- Estructura de oraciones

In [5]:
# Leer el archivo y procesarlo con spaCy
with open('./percy_jackson_book_1.txt', encoding='utf-8') as file:
    doc = nlp(file.read())

print(f"Documento procesado: {len(doc)} tokens")

Documento procesado: 127898 tokens


### 4.1 Vista Previa del Documento

Mostramos los primeros 50 tokens para verificar que el documento se cargó correctamente.

In [6]:
# Primeros 50 tokens del documento
doc[:50]



BOOKS BY RICK RIORDAN

PERCY JACKSON AND THE OLYMPIANS
The Lightning Thief
The Sea of Monsters
The Titan’s Curse
The Battle of the Labyrinth
The Last Olympian
The Demigod Files
Percy Jackson’s Greek Gods, illustrated by John Rocco

## 5. Búsqueda de Patrones con Matcher

Usamos `Matcher` de spaCy para encontrar la frase "peanut butter" en el documento.

**¿Cómo funciona?**
- Definimos un patrón: dos tokens consecutivos con texto "peanut" y "butter" (en minúsculas)
- El Matcher busca todas las ocurrencias en el documento
- Devuelve tuplas `(match_id, start, end)` con la posición de cada coincidencia

In [7]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{'LOWER': 'peanut'}, {'LOWER': 'butter'}]
matcher.add("like", [pattern])

found_matches = matcher(doc)
found_matches

[(18194338103975822726, 1326, 1328), (18194338103975822726, 1551, 1553)]

### 5.1 Contexto de las Coincidencias

Veamos el contexto alrededor de cada coincidencia (tokens antes y después) para entender mejor el uso.

In [8]:
start, end = found_matches[0][1:]
doc[start-9:end+13]

the back of the


head with chunks of peanut butter-and-ketchup sandwich.
Grover was an easy target.

In [9]:
start, end = found_matches[1][1:]
doc[start-7:end+5]

“It’s okay. I like peanut butter.”
He dodged

### 5.2 Oraciones Completas

Extraemos las oraciones completas que contienen las coincidencias. Esto proporciona el contexto completo y gramaticalmente correcto.

In [10]:
# Primero obtenemos todas las oraciones del documento
sentences = list(doc.sents)

print(f"Oraciones que contienen 'peanut butter':\n")

# Buscamos cada coincidencia en las oraciones
for i, (_, start, end) in enumerate(found_matches, 1):
    for sentence in sentences:
        # Verificar si la coincidencia está dentro de esta oración
        if sentence.start <= start and sentence.end >= end:
            print(f"{i}. {sentence.text.strip()}\n")

Oraciones que contienen 'peanut butter':

1. All the way into the city, I put up with Nancy Bobofit, the freckly,
redheaded kleptomaniac girl, hitting my best friend Grover in the back of the

head with chunks of peanut butter-and-ketchup sandwich.

2. I like peanut butter.”

